# Fine-tuning a model with the Trainer API
# 使用Trainer微調模型

 Transformers 提供了一個`Trainer`類別來**微調**資料集上提供的任何預訓練模型。完成資料預處理工作後，只需執行幾個步驟即可定義Trainer, 準備運行環境`Trainer.train()`

安裝套件

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate -U #安裝所需的庫。

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (1

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding #創建了一個 DataCollatorWithPadding 實例

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint) #使用 tokenizer 將數據樣本批次化並進行填充


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True) #進行標記化


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

### 定義`Trainer`的參數
我們定義我們之前的第一步`Trainer`是定義一個類別，其中包含將用於訓練和評估的`TrainingArguments`所有超參數。為求簡單，這邊`Trainer`提供的參數是儲存訓練模型的目錄以及訓練的輪數。對於其餘所有內容，保留預設值，這對於基本的微調應該非常有效。

In [3]:
from transformers import TrainingArguments #創建了一個 TrainingArguments 物件 training_args

training_args = TrainingArguments("test-trainer", num_train_epochs=10) #訓練過程中的訓練周期數

使用`AutoModelForSequenceClassification`有兩個標籤的類別

In [4]:
from transformers import AutoModelForSequenceClassification #創建一個序列分類模型

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) #指定了模型的輸出類別數量

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 說明
在實例化這個預訓練模型後會收到一個警告。這是因為BERT沒有在對句子對進行分類的任務上進行預訓練，因此預訓練模型的頭部被丟棄了(bert-base-uncase)，並且加入了一個適合序列分類的新頭部。表示有些權重未被使用（對應於被丟棄的預訓練頭部的那些）以及一些其他權重是隨機初始化的（新頭部）

### 定義`Trainer`
有了模型，就可以傳入建構的所有對象來定義一個訓練器模型、`training_args`、訓練和驗證資料集、`data_collator`和分詞器

In [5]:
from transformers import Trainer
# 創建一個 Trainer 實例
trainer = Trainer(
    model, # 設置訓練的模型
    training_args, # 設置訓練的參數，例如訓練周期、優化器等
    train_dataset=tokenized_datasets["train"], # 設置訓練數據集
    eval_dataset=tokenized_datasets["validation"], # 設置驗證數據集
    data_collator=data_collator, # 設置數據收集器，用於將樣本批次化並進行填充
    tokenizer=tokenizer, # 設置標記器，用於處理數據樣本
)

### 說明
這樣傳遞分詞器時，訓練器使用的預設`data_collator`將是之前定義的`DataCollatorWithPadding`，因此可以跳過`data_collator=data_collator`這一行

### 開始訓練
呼叫`Trainer`中的`train()`方法

In [7]:
trainer.train() #開始了模型的訓練過程

Step,Training Loss


KeyboardInterrupt: 

### 計算模型表現的度量方法
- 計算408個預測中，有幾個是和真值一模一樣的

In [ ]:
import evaluate
import numpy as np

predictions = trainer.predict(tokenized_datasets["validation"]) # 使用訓練器對驗證數據集進行預測
preds = np.argmax(predictions.predictions, axis=-1) # 從預測中獲取類別的索引
predictions = trainer.predict(tokenized_datasets["validation"]) # 再次使用訓練器對驗證數據集進行預測
metric = evaluate.load("glue", "mrpc") # 載入 GLUE 數據集的評估指標
metric.compute(predictions=preds, references=predictions.label_ids)# 計算模型的性能指標


{'accuracy': 0.8357843137254902, 'f1': 0.8838821490467937}